# Activity: Build an LSTM to build a classifier for the Human Activity Recognition Using Smartphones Dataset #

Adapted from https://github.com/servomac/Human-Activity-Recognition/

Dataset information at https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones#





**If you have not yet installed wget, you will need to start by doing so:**

1) Open a new terminal/command window
2) Type `conda install python-wget`

In [ ]:
#All our imports
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from matplotlib.pyplot import *

This next cell uses "wget" to download data from an online archive. You'll only need to run this the first time you run the notebook, because when you run this the first time it will save the data in the directory with this notebook.

In [10]:
import wget
import os
import zipfile


#This is going to take a little while
if not os.path.isdir("UCI HAR Dataset"):
    print("downloading")
    wget.download("http://archive.ics.uci.edu/ml/machine-learning-databases/00240/UCI HAR Dataset.zip")
    
    with zipfile.ZipFile("UCI HAR Dataset.zip", 'r') as zip_ref:
        zip_ref.extractall("")
        
    print("done")
    
else:
    print("you already have it!")

you already have it!


In [ ]:
# This is helper code from the github repo.
#No need to edit this, but it's helpful to glance through this code to see what it does

DATADIR = 'UCI HAR Dataset' 

SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}


def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).values
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).values

def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test


def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

def _count_classes(y):
    return len(set([tuple(category) for category in y]))

## Our LSTM model ##

This is where the important stuff happens. Try to understand everything happening here, and experiment with changing it once you've got the basic method working.


In [ ]:
#Load the data and splits into training and test sets, where X is input and Y is output
X_train, X_test, Y_train, Y_test = load_data()

#Compute some information from the data, which we need to set up the model
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

In [ ]:
#After loading data, it's always a good idea to get a feel for what is in the data 

#For instance, we can use np.shape() to look at the shape of our examples
print(np.shape(X_train)) #this shows us we have 7352 training examples, where each input is 128x9 (9 channels of 128 samples each)

In [ ]:
#We can peek at an individual example
exampleNumber = 0
channelNumber = 3
plot(np.transpose(X_train[exampleNumber])[channelNumber])

In [ ]:
print(np.shape(Y_train)) #and we have a 6-dimensional output for each one of our 7352 examples

In [ ]:
#We're using a "one-hot" encoding, which means that "1 0 0 0 0 0" means "class 1", "0 1 0 0 0 0" means class 2, and so on
print(Y_train[0]) #Prints the 6-dimensional output for the first training example

In [ ]:
# This is where things get fun.

# Set some parameters to control training
# The number of epochs is the number of passes through the training data to make before you stop training
epochs = 5 # Play with this; might especially want to increase it once you know things are basically working

# The batch size is the number of examples to use to compute the gradient before updating the weights
# If batch_size is the same as the training set size, we use the whole training set every time. This is called "batch gradient descent"
# If batch_size is 1 then this is "stochastic gradient descent"
# If batch_size is something in between, then this is called "mini-batch" gradient descent
# Each choice will have different tradeoffs in terms of training time and ultimate accuracy, but there's no "right way" to do it
# For more info, see https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/#:~:text=The%20batch%20size%20is%20a%20hyperparameter%20that%20defines%20the%20number,updating%20the%20internal%20model%20parameters.&text=When%20the%20batch%20size%20is,called%20mini%2Dbatch%20gradient%20descent. 
batch_size = 16 #You might want to play with this too

#How many hidden neurons do we want?
n_hidden = 32 #Feel free to change this


#This is the Keras code for specifying the network architecture:
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim))) #an LSTM layer
model.add(Dropout(0.5)) #This is a method to try to prevent overfitting. See more at https://machinelearningmastery.com/dropout-regularization-deep-learning-models-keras/
model.add(Dense(n_classes, activation='sigmoid')) #a plain "dense" layer comes after the LSTM layer

#This line specifies how to train this network.
#You might want to play with these 3 values
#see more at https://keras.io/api/losses/, https://keras.io/api/optimizers/, https://keras.io/api/metrics/
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

#This line of code actually trains the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

# Evaluate the model on the test data
print(confusion_matrix(Y_test, model.predict(X_test)))

## Next steps: Experiment!

### 1. Try digging into & tweaking the results above. ###

* What happens with a different optimizer?
* Can you edit the code to use a validation set with early stopping, as well?
* What happens to the training accuracy, test/validation accuracy, and training time if you change the number of training examples?
* What happens if you change the dropout?
* Can you visualise this data (e.g., in Python using matplotlib, or in Excel) to get a sense of what it looks like?
* What else might you print each epoch, instead of just accuracy and loss?
* What else might you do if you were trying to release a commercial product that did this classification? How would you test it? How else might you edit the code to train your neural network? What would your main considerations be in deploying it?

### 2. Do more reading about all the options available to you in Keras ###
* https://keras.io/api/optimizers/
* https://keras.io/api/metrics/
* https://keras.io/api/losses/
* Use the `?` operation in Python to learn more about your options in code (e.g., `?model.compile`)
